In [1]:
import os
os.environ["HF_HUB_CACHE"] = "/media/user/8a594cab-20d9-43ef-8d0e-b60b5cf43462/huggingface_models/hub"
#https://huggingface.co/docs/transformers/installation?highlight=transformers_cache#cache-setup
#os.environ["HF_HOME"] = "/media/user/8a594cab-20d9-43ef-8d0e-b60b5cf43462/huggingface_models"
model_name="avichr/Legal-heBERT"#'meta-llama/Llama-2-70b-chat-hf'

In [2]:
import faiss
import torch
from transformers import AutoTokenizer, BertModel

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=BertModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
with torch.no_grad():
    model.pooler.dense.weight*=0#torch.ones(model.pooler.dense.weight.shape)
    model.pooler.dense.biass=0#torch.zeros(model.pooler.dense.biass.shape)

In [5]:
import datasets
data = datasets.load_dataset('LevMuchnik/SupremeCourtOfIsrael')

Found cached dataset parquet (/home/user/.cache/huggingface/datasets/LevMuchnik___parquet/LevMuchnik--SupremeCourtOfIsrael-7f46ab7220db2581/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import pandas as pd
import re
from tqdm.notebook import tqdm

In [7]:
#from concurrent.futures import ProcessPoolExecutor

# filter

In [7]:
texts=data['train']['text']

In [11]:
def get_paragraphs(text_list):
    ans = []
    # Regex to match paragraph separators
    paragraph_separator = re.compile(r'(\n\s*)+\n')
    bads = (
            re.compile(r'<!--\s*\n\s*/\*\s*Font Definitions\s*\*/'),
            re.compile(r'\uFFFD'),
            re.compile(r'endstream.*?(?=\n|$)'),
            re.compile(r'/Author \(user\)\n|/Creator'),
           )
            

    
    for doc_idx, text in enumerate(text_list):
        if not text:
            continue

        # Splitting text into paragraphs using the defined regex
        paragraphs = paragraph_separator.split(text)

        count = 0
        for paragraph in paragraphs:
            skip=False
            for b in bads:
                if b.search(paragraph):
                    skip=True
                    break

            if skip:
                continue

            paragraph = paragraph.strip()  # Stripping whitespace
            # Only add non-empty paragraphs
            if paragraph:
                ans.append({'doc': doc_idx, 'count': count, 'text': paragraph})
                count += 1

    return ans

In [12]:
pars=get_paragraphs(tqdm(texts))

  0%|          | 0/751194 [00:00<?, ?it/s]

In [ ]:
len(pars)

# tokenizing

In [15]:
#os.environ["TOKENIZERS_PARALLELISM"]='false'

In [ ]:
tokens=list(map(tokenizer.encode,tqdm([d['text'] if d['text'] else '' for d in pars])))
len(tokens)

In [ ]:
len(tokens)

In [ ]:
import gc

In [ ]:
for t,p in zip(tokens,pars):
    p['tokens']=t

del(tokens)
gc.colect()

In [ ]:
token_chunks=[]
for p in tqdm(pars):
    if not p:
        continue
    for i,toks_index in enumerate(range(0,len(p['tokens']),model.config.max_position_embeddings)):
        toks=p['tokens'][toks_index:toks_index+model.config.max_position_embeddings]
        cur={k:p[k] for k in ('doc', 'count')}
        cur.update({'chunk':i,'tokens':toks})
        #cur['chunk']=i
        #cur['tokens']=toks
        token_chunks.append(cur)

In [ ]:
#torch.save(token_chunks,'clean_data.pt')

In [ ]:
token_chunks[3]#.sort(key=lambda d:len(d['tokens']))

In [ ]:
token_chunks.sort(key=lambda d:len(d['tokens']))

In [ ]:
torch.save(token_chunks,'clean_data.pt')

# embed

In [8]:
token_chunks=torch.load('clean_data.pt')

In [9]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [10]:
def padded_batch(tokens):
    lens=[len(t) for t in tokens]
    m=max(lens)
    toks=torch.IntTensor([t+(m-l)*[0] for l,t in zip(lens,tokens)])#,device=model.device)
    mask=torch.IntTensor([l*[1]+(m-l)*[0] for l in lens])#,device=model.device)
    return {'input_ids':toks,'attention_mask':mask}

In [11]:
all_embedding=[]
batch=256
with torch.no_grad():
    for i in tqdm(range(0,len(token_chunks),batch)):
        ld=token_chunks[i:i+batch]
        toks=[d['tokens'] for d in ld]
        inputs=padded_batch(toks)
        inputs={k:v.to('cuda') for  k,v in inputs.items()}
        outputs=model(**inputs)
        outputs=outputs.last_hidden_state.mean([1]).cpu()
        all_embedding.extend(outputs)

  0%|          | 0/97606 [00:00<?, ?it/s]

In [15]:
import gc
del(token_chunks)
gc.collect()

753

In [ ]:
torch.save(all_embedding,'chunked_emb.pt')

In [ ]:
emb=torch.stack(all_embedding)
emb.shape